# World Model

## Imports


In [48]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
import joblib
import os

## Load Dataset

In [49]:
def load_data(filepath="../dataset/dataset_limited_2.txt"):
    """Loads the dataset using pandas."""
    try:
        df = pd.read_csv(filepath)
        print(f"Dataset loaded successfully. Shape: {df.shape}")
        df = df.dropna()
        print(f"Shape after dropping NaNs: {df.shape}")
        return df
    except FileNotFoundError:
        print(f"Error: Dataset file not found at {filepath}")
        return None
    except Exception as e:
        print(f"Error loading dataset: {e}")
        return None

In [50]:
# 1. Load the dataset
dataframe = load_data()

Dataset loaded successfully. Shape: (1818, 14)
Shape after dropping NaNs: (1818, 14)


In [51]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1818 entries, 0 to 1817
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   distance_red_init     1818 non-null   float64
 1   angle_red_init        1818 non-null   float64
 2   distance_green_init   1818 non-null   float64
 3   angle_green_init      1818 non-null   float64
 4   distance_blue_init    1818 non-null   float64
 5   angle_blue_init       1818 non-null   float64
 6   rSpeed                1818 non-null   int64  
 7   lSpeed                1818 non-null   int64  
 8   distance_red_final    1818 non-null   float64
 9   angle_red_final       1818 non-null   float64
 10  distance_green_final  1818 non-null   float64
 11  angle_green_final     1818 non-null   float64
 12  distance_blue_final   1818 non-null   float64
 13  angle_blue_final      1818 non-null   float64
dtypes: float64(12), int64(2)
memory usage: 199.0 KB


## Preprocess Dataset

In [52]:
def prepare_data(df):
    """Separates features (X) and target variables (Y)."""
    # Input Features: initial state (6) + action (2) = 8 features
    X = df.iloc[:, :8].values
    # Target Variables: final state (6) = 6 features
    Y = df.iloc[:, 8:].values
    print(f"Features (X) shape: {X.shape}")
    print(f"Targets (Y) shape: {Y.shape}")
    return X, Y

In [53]:
def split_data(X, Y, test_size=0.2, random_state=42):
    """Splits data into training and testing sets."""
    X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, test_size=test_size, random_state=random_state
    )
    print(f"Training set size: {X_train.shape[0]} samples")
    print(f"Testing set size: {X_test.shape[0]} samples")
    return X_train, X_test, Y_train, Y_test

In [54]:
# 2. Prepare Data
X, Y = prepare_data(dataframe)

Features (X) shape: (1818, 8)
Targets (Y) shape: (1818, 6)


In [55]:
def scale_features(X_train, X_test):
    """Scales input features using StandardScaler."""
    
    scaler = StandardScaler()
    
    # Fit scaler ONLY on training data
    X_train_scaled = scaler.fit_transform(X_train)
    
    # Transform both train and test data
    X_test_scaled = scaler.transform(X_test)
    
    print("Features scaled.")
    
    return X_train_scaled, X_test_scaled, scaler # Return scaler to save it

In [56]:
# 3. Split Data
X_train, X_test, Y_train, Y_test = split_data(X, Y)

# 4. Scale Features (Important!)
X_train_scaled, X_test_scaled, scaler = scale_features(X_train, X_test)

Training set size: 1454 samples
Testing set size: 364 samples
Features scaled.


## Train model

In [63]:
# from tqdm import tqdm
import time
from sklearn.model_selection import GridSearchCV

def train_model_with_gridsearch(X_train, Y_train):
    """Trains a RandomForestRegressor using GridSearchCV for hyperparameter tuning."""
    print("\n--- Starting GridSearchCV for RandomForestRegressor ---")

    # Define the parameter grid to search
    # Start with a smaller grid, then expand if needed and time permits
    param_grid = {
        'n_estimators': [100, 200, 300, 500, 1000, 2000],            # Number of trees
        'max_depth': [5, 10, 20, 30, None],           # Max depth of trees (None means no limit initially)
        'min_samples_split': [2, 5, 10, 20],         # Min samples to split a node
        'min_samples_leaf': [1, 3, 5, 10],          # Min samples in a leaf node
        'max_features': ['sqrt', 'log2'],       # Number of features to consider for split
        'oob_score': [True] # Can include if you want OOB score for the final best model
    }

    # Initialize the base model
    rf = RandomForestRegressor(random_state=42, n_jobs=-1) # Use n_jobs in base for fitting final model too

    # Initialize GridSearchCV
    # Scoring: Use negative MAE because GridSearchCV maximizes score. Lower MAE is better.
    # cv=5: 5-fold cross-validation
    # verbose=2: Print progress updates
    # n_jobs=-1: Use all available CPU cores for the search
    grid_search = GridSearchCV(estimator=rf,
                               param_grid=param_grid,
                               cv=5,
                               scoring='neg_mean_absolute_error', # Or 'neg_mean_squared_error'
                               n_jobs=-1,
                               verbose=3) # Increase verbosity to see progress

    print(f"Searching grid: {param_grid}")
    start_time = time.time()

    # Fit GridSearchCV to the data
    grid_search.fit(X_train, Y_train)

    end_time = time.time()
    print(f"GridSearchCV finished in {end_time - start_time:.2f} seconds.")

    # Print the best parameters found
    print("\nBest parameters found by GridSearchCV:")
    print(grid_search.best_params_)

    # Print the best score (negative MAE or MSE)
    print(f"\nBest cross-validation score (Negative MAE): {grid_search.best_score_:.4f}")
    print(f"Corresponds to MAE: {-grid_search.best_score_:.4f}")


    # The best model found by GridSearchCV is stored in best_estimator_
    best_model = grid_search.best_estimator_
    print("\nBest model training complete.")

    # You can optionally check OOB score if it was calculated
    # hasattr checks if the attribute exists before accessing
    if hasattr(best_model, 'oob_score_') and best_model.oob_score_:
        print(f"Best Model OOB score: {best_model.oob_score_:.4f}")

    return best_model

In [64]:
# 5. Train Model using GridSearchCV (using scaled data)
if X_train_scaled is not None:
    world_model = train_model_with_gridsearch(X_train_scaled, Y_train)
else:
    print("Skipping model training due to data scaling issues.")
    exit()


--- Starting GridSearchCV for RandomForestRegressor ---
Searching grid: {'n_estimators': [100, 200, 300, 500, 1000, 2000], 'max_depth': [5, 10, 20, 30, None], 'min_samples_split': [2, 5, 10, 20], 'min_samples_leaf': [1, 3, 5, 10], 'max_features': ['sqrt', 'log2'], 'oob_score': [True]}
Fitting 5 folds for each of 960 candidates, totalling 4800 fits
[CV 1/5] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100, oob_score=True;, score=-51.860 total time=   0.3s
[CV 5/5] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100, oob_score=True;, score=-52.332 total time=   0.3s
[CV 2/5] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100, oob_score=True;, score=-51.303 total time=   0.3s
[CV 4/5] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100, oob_score=True;, score=-53.095 total time=   0.3s
[CV 3/5] END max_depth=5, max_fea

/Users/albertolandi/anaconda3/envs/IR2/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV 5/5] END max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=500, oob_score=True;, score=-26.802 total time=   2.3s
[CV 4/5] END max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=2000, oob_score=True;, score=-26.525 total time=  13.1s
[CV 5/5] END max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=2000, oob_score=True;, score=-25.894 total time=  13.4s
[CV 1/5] END max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=1000, oob_score=True;, score=-27.280 total time=   4.6s
[CV 2/5] END max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=1000, oob_score=True;, score=-27.045 total time=   4.5s
[CV 3/5] END max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=1000, oob_score=True;, score=-27.023 total time=   4.4s
[CV 4/5] END max_depth=20, max_features=sqrt, min_samples_leaf=1,

## Evaluate

In [65]:
def evaluate_model(model, X_test, Y_test):
    """Evaluates the model using MAE and MSE."""
    Y_pred = model.predict(X_test)

    mae = mean_absolute_error(Y_test, Y_pred)
    mse = mean_squared_error(Y_test, Y_pred)
    rmse = np.sqrt(mse) # Root Mean Squared Error

    print("\n--- Model Evaluation ---")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"Mean Squared Error (MSE):  {mse:.4f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

    # Optional: Print metrics per output feature
    print("\nMAE per output feature:")
    output_features = [
        'dist_red_final', 'angle_red_final', 'dist_green_final',
        'angle_green_final', 'dist_blue_final', 'angle_blue_final'
    ]
    for i, name in enumerate(output_features):
         mae_feature = mean_absolute_error(Y_test[:, i], Y_pred[:, i])
         print(f"  {name}: {mae_feature:.4f}")

    return mae, mse

In [68]:
# 6. Evaluate the Best Model found by GridSearch (using scaled test data)
if world_model is not None:
    evaluate_model(world_model, X_test_scaled, Y_test)


--- Model Evaluation ---
Mean Absolute Error (MAE): 24.3979
Mean Squared Error (MSE):  1798.4146
Root Mean Squared Error (RMSE): 42.4077

MAE per output feature:
  dist_red_final: 40.7431
  angle_red_final: 8.4946
  dist_green_final: 36.7146
  angle_green_final: 18.2790
  dist_blue_final: 38.3186
  angle_blue_final: 3.8372


## Save model

In [69]:
def save_model_and_scaler(model, scaler, model_filename="world_model_v3.joblib", scaler_filename="scaler_v3.joblib"):
    """Saves the trained model and scaler to disk."""
    try:
        # Ensure the directory exists
        model_dir = "../src/models"
        os.makedirs(model_dir, exist_ok=True)

        model_path = os.path.join(model_dir, model_filename)
        scaler_path = os.path.join(model_dir, scaler_filename)

        joblib.dump(model, model_path)
        joblib.dump(scaler, scaler_path)
        print(f"Model saved to {model_path}")
        print(f"Scaler saved to {scaler_path}")
    except Exception as e:
        print(f"Error saving model/scaler: {e}")

In [70]:
# 7. Save Model and Scaler
save_model_and_scaler(world_model, scaler)

Model saved to ../src/models/world_model_v3.joblib
Scaler saved to ../src/models/scaler_v3.joblib


In [ ]:
# Example prediction (how you'd use it later)
print("\n--- Example Prediction ---")

# Take the first sample from the original test set
sample_X = X_test[0].reshape(1, -1)
sample_Y_actual = Y_test[0]

# Scale the sample using the *saved* scaler
sample_X_scaled = scaler.transform(sample_X)

# Predict using the trained model
sample_Y_pred = world_model.predict(sample_X_scaled)

print(f"Input State + Action: {sample_X[0]}")
print(f"Actual Final State:   {sample_Y_actual}")
print(f"Predicted Final State:{sample_Y_pred[0]}")


--- Example Prediction ---
Input State + Action: [ 4.10674455e+02  1.22678632e+02  1.39104272e+03 -1.74541181e+02
  8.04670468e+02  5.43256407e+01  1.00000000e+00 -6.00000000e+00]
Actual Final State:   [ 419.68603568  120.04972943 1377.94257775 -175.22419136  825.72434115
   54.38562828]
Predicted Final State:[ 418.00912156  123.23844485 1385.85273925 -173.32613874  813.60056139
   55.5324251 ]
